## Validate HED in a BIDS dataset that uses library schema.

Validating annotations HED as you develop them makes the annotation process much easier and
faster to debug. This notebook validates HED in a BIDS dataset.

The tool creates a `BidsDataset` object, which represents the information from a BIDS
dataset that is relevant to HED, including the `dataset_description.json`,
all `events.tsv` files, and all `events.json` sidecar files.

The `validate` method of `BidsDataset` first validates all of the `events.json` sidecars
and then assembles the relevant sidecars for each `events.tsv` file and validates it.
The validation uses the HED schemas specified in the `HEDVersion` field of the
dataset's `dataset_description.json` file.

The script does the following steps:

1. Set the dataset location (`bids_root_path`) to the absolute path of the root of your BIDS dataset.
2. Indicates whether to check for warnings during validation (`check_for_warnings`).
3. Create a `BidsDataset` for the dataset.
4. Validate the dataset and output the issues.

**Note:** This validation pertains to event files and HED annotation only. It does not do a full BIDS validation.

The example below uses a
[small version](https://github.com/hed-standard/hed-examples/tree/main/datasets/eeg_ds003654s_hed)
of the Wakeman-Hanson face-processing dataset available on openNeuro as
[ds003654](https://openneuro.org/datasets/ds003645/versions/2.0.0).

This dataset has no validation errors, but since we have set `check_for_warnings` to `True`,
validation returns warnings that the `sample` column does not have any metadata.

For validation of a single `events.json` files during annotation development,
users often find the [online sidecar tools](https://hedtools.ucsd.edu/hed/sidecar)
convenient, but the online tool does not provide complete dataset-level validation.

In [1]:
import os
from hed.errors import get_printable_issue_string
from hed.schema import HedSchemaGroup, load_schema, load_schema_version
from hed.tools import BidsDataset

## Set the dataset location and the check_for_warnings flag
check_for_warnings = False
bids_path = os.path.realpath('../../../datasets/eeg_ds003654s_hed_library')

## Validate the dataset using the information from the dataset_description
print("Handling a BIDS data set that uses library schema")
bids = BidsDataset(bids_path)
issue_list1 = bids.validate(check_for_warnings=check_for_warnings)
if issue_list1:
    issue_str1 = get_printable_issue_string(issue_list1, "HED validation errors: ", skip_filename=False)
else:
    issue_str1 = "No HED validation errors"
print(issue_str1)


## Now validate using prerelease schema
print("\nNow validating with the prerelease schema.")
base_version = '8.0.0'
library1_url = "https://raw.githubusercontent.com/hed-standard/hed-schema-library/main/" + \
               "library_schemas/score/hedxml/HED_score_0.0.1.xml"
library2_url = "https://raw.githubusercontent.com/hed-standard/hed-schema-library/main/" + \
               "library_schemas/testlib/hedxml/HED_testlib_1.0.2.xml"
schema_list = [load_schema_version(xml_version=base_version)]
schema_list.append(load_schema(library1_url, library_prefix="sc"))
schema_list.append(load_schema(library2_url, library_prefix="test"))
bids.schemas = HedSchemaGroup(schema_list)
issue_list2 = bids.validate(check_for_warnings=check_for_warnings)
if issue_list2:
    issue_str2 = get_printable_issue_string(issue_list2, "HED validation errors: ", skip_filename=False)
else:
    issue_str2 = "No HED validation errors"
print(issue_str2)

Handling a BIDS data set that uses library schema
No HED validation errors

Now validating with the prerelease schema.
No HED validation errors
